In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import time
from ar1_approx import integrand
from ar1_approx import addacooper
import numpy as np
import scipy.stats as st
from scipy.stats import norm
import scipy.integrate as integrate
import numba

# to print plots inline
%matplotlib inline

In [2]:
# specify parameters
alpha_k = 0.29715
alpha_l = 0.65
delta = 0.154
psi = 1.08
w = 0.7
r= 0.04
sigma_z = 0.213
mu = 0.0
rho = 0.7605
sizez = 9
zz=1
betafirm = (1 / (1 + r))
dens = 5

In [3]:
# put in bounds here for the capital stock space
kstar = ((((1 / betafirm - 1 + delta) * ((w / alpha_l) **
                                         (alpha_l / (1 - alpha_l)))) /
         (alpha_k * (zz ** (1 / (1 - alpha_l))))) **
         ((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
k_grid = K[::-1]
sizek = k_grid.shape[0]

ln_z_grid, pi = addacooper(sizez, mu, rho, sigma_z)
z_grid = np.exp(ln_z_grid)
print(z_grid)

[ 0.57170087  0.72611947  0.82344962  0.91128377  1.          1.09735302
  1.21440337  1.37718384  1.74916648]


In [4]:
def quad_adjcost(k, k_prime):
    
    return (psi/2) * (((k_prime - (1-delta)*k) / k)**2) * k

In [5]:
def op_profits(K_grid, Z_grid, sizek, sizez):
    
    op = np.zeros((sizek, sizez))
    
    for i, k in enumerate(K_grid):
        for j, z in enumerate(Z_grid):
            op[i,j] = (1-alpha_l) * ((alpha_l/w)**(alpha_l/(1-alpha_l))) * (z**(1/(1-alpha_l))) * (k**(alpha_k/(1-alpha_l)))
    return op
        
def cash_flow(K_grid, Z_grid, sizek, sizez):
    
    e = np.zeros((sizek, sizez, sizek))
    
    for i, k in enumerate(K_grid):
        for j, z in enumerate(Z_grid):
            for h, k_prime in enumerate(K_grid):
                e[i,j,h] = op[i,j] - (k_prime - (1-delta)*k) - quad_adjcost(k, k_prime)
    
    return e